In [1]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=c186b3e91485ee79c1caf7dd7d4f256f77c28eed10b9387b626ea1b59b2a9c77
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [2]:
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import wave
import librosa
from python_speech_features import *
import re
from tensorflow_hub import load, Module
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input, LSTM, Bidirectional
from tensorflow.keras.models import Model
from keras.utils import to_categorical
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error

In [3]:
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/data_ML5/

Mounted at /content/drive
/content/drive/MyDrive/data_ML5


In [4]:
train_df = pd.read_csv('train_split_Depression_AVEC2017.csv')
test_df = pd.read_csv('dev_split_Depression_AVEC2017.csv')

In [34]:
train_id = train_df[['Participant_ID']]['Participant_ID'].tolist()
test_id = test_df[['Participant_ID']]['Participant_ID'].tolist()
# train_label = train_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
# test_label = test_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
train_label = train_df[['PHQ8_Score']]['PHQ8_Score'].tolist()
test_label = test_df[['PHQ8_Score']]['PHQ8_Score'].tolist()

In [6]:
elmo = load("https://tfhub.dev/google/elmo/3").signatures["default"]

In [36]:
# training set
train_features = []
train_targets = []

# test set
test_features = []
test_targets = []

features_whole = []
targets_whole = []
whole_split_num = train_id + test_id
whole_targets = train_label + test_label

counter_train = 0
counter_test = 0
debt = 0

In [10]:
topics = []
with open('questions.txt', 'r') as f:
    for line in f.readlines():
        topics.append(line.strip('\n').strip())

In [37]:
def identify_topics(sentence):
    sentence = re.sub(r'\(|\)', '', sentence)
    pattern = r'\b(what|how|where|when|why|are|do|have|who|who\'s|what\'s|why\'d|what\'d)\b(.*)$'
    match = re.search(pattern, sentence, re.IGNORECASE)
    if match:
        question = match.group(0).strip()
        if question in topics:
          return True
    return False



def extract_features(number, features, target, targets, mode):
    print(number)
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')

    wavefile = wave.open('{0}_AUDIO.wav'.format(number, 'r'))
    framerate = wavefile.getframerate()
    nframes = wavefile.getnframes()
    wave_data = np.frombuffer(wavefile.readframes(nframes), dtype=np.short)

    time_range = []
    responses = []
    response = ''
    response_flag = False
    start_time = 0
    stop_time = 0
    signal = []

    global counter_train, counter_test

    for row in transcript.itertuples():
        if row.speaker == 'Ellie':

            content = row.value.strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    responses.append(response.strip())
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    responses.append(response)
                response = ''
        elif row.speaker == 'Participant':
            if 'scrubbed_entry' in row.value:
                continue
            elif response_flag:
                response +=' ' +row.value.split('\n')[0].strip()
            start_time = int(row.start_time*framerate)
            stop_time = int(row.stop_time*framerate)
            signal = np.hstack((signal, wave_data[start_time:stop_time].astype(np.float64)))
    clip = framerate*1*15
    if len(responses) == 0:
      print("Empty")
    else:
      text_feature = elmo(tf.constant(responses))["elmo"]
      text_feature = tf.reduce_mean(text_feature, axis=0)
 #      if target == 1 and mode == 'train':
      if target >= 10 and mode == 'train':
        times = 3 if counter_train < 48 else 2
        for i in range(times):
              melspec = librosa.feature.melspectrogram(y=signal[clip*i:clip*(i+1)], n_mels=80,sr=framerate)
              text_f = text_feature[i*10:(i+1)*10]
              features.append([text_f.numpy(), melspec])
              targets.append(target)
              counter_train+=1
      else:
          melspec = librosa.feature.melspectrogram(y=signal[:clip], n_mels=80, sr=framerate)
          text_f = text_feature[:10]
          features.append([text_f.numpy(), melspec])
          targets.append(target)

In [38]:
def extract_features_whole(number, features, target, targets):
    print(number)
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')

    wavefile = wave.open('{0}_AUDIO.wav'.format(number, 'r'))
    framerate = wavefile.getframerate()
    nframes = wavefile.getnframes()
    wave_data = np.frombuffer(wavefile.readframes(nframes), dtype=np.short)

    time_range = []
    responses = []
    response = ''
    response_flag = False
    start_time = 0
    stop_time = 0
    signal = []

    global counter_train, counter_test

    for row in transcript.itertuples():
        if row.speaker == 'Ellie':

            content = row.value.strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    responses.append(response.strip())
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    responses.append(response)
                response = ''
        elif row.speaker == 'Participant':
            if 'scrubbed_entry' in row.value:
                continue
            elif response_flag:
                response +=' ' +row.value.split('\n')[0].strip()
            start_time = int(row.start_time*framerate)
            stop_time = int(row.stop_time*framerate)
            signal = np.hstack((signal, wave_data[start_time:stop_time].astype(np.float64)))
    clip = framerate*1*15
    if len(responses) == 0:
      print("Empty")
    else:
      text_feature = elmo(tf.constant(responses))["elmo"]
      text_feature = tf.reduce_mean(text_feature, axis=0)
 #     if target == 1:
      if target >= 10:
        times = 3 if counter_train < 48 else 2
        for i in range(times):
              melspec = librosa.feature.melspectrogram(y=signal[clip*i:clip*(i+1)], n_mels=80,sr=framerate)
              features.append([text_feature[i*10:(i+1)*10], melspec])
              targets.append(target)
              counter_train+=1
      else:
          melspec = librosa.feature.melspectrogram(y=signal[:clip], n_mels=80, sr=framerate)
          features.append([text_feature[:10], melspec])
          targets.append(target)

In [40]:
for i in range(len(train_id)):
    extract_features(train_id[i], train_features, train_label[i], train_targets, 'train')

303
304
305
310
312
313
315
316
317
318
319
320
321
322
324
325
326
327
328
330
333
336
338
339
340
341
343
344
345
347
348
350
351
352
353
355
356
357
358
360
362
363
364
366
368
369
370
371
372
374
375
376
379
380
383
385
386
391
392
393
397
400
401
402
409
412
414
415
416
419
423
425
426
427
428
429
430
433
434
437
441
443
444
445
446
447
448
449
454
455
456
457
459
463
464
468
471
473
474
475
478
479
485
486
487
488
491


In [ ]:
np.savez('fuse_train_samples_regression.npz', train_features)
np.savez('fuse_train_labels_regression.npz', train_targets)

In [43]:
for i in range(len(test_id)):
    extract_features(test_id[i], test_features, test_label[i], test_targets, 'test')

print(np.shape(train_targets), np.shape(test_targets))

302
307
331
335
346
367
377
381
382
388
389
390
395
403
404
406
413
417
418
420
422
436
439
440
451
Empty
458
Empty
472
476
477
482
483
484
489
490
492
(154,) (33,)


In [ ]:
np.savez('fuse_test_samples_regression.npz', test_features)
np.savez('fuse_test_labels_regression.npz', test_targets)

In [46]:
for index in range(len(whole_split_num)):
  extract_features_whole(whole_split_num[index], features_whole, whole_targets[index], targets_whole)

303
304
305
310
312
313
315
316
317
318
319
320
321
322
324
325
326
327
328
330
333
336
338
339
340
341
343
344
345
347
348
350
351
352
353
355
356
357
358
360
362
363
364
366
368
369
370
371
372
374
375
376
379
380
383
385
386
391
392
393
397
400
401
402
409
412
414
415
416
419
423
425
426
427
428
429
430
433
434
437
441
443
444
445
446
447
448
449
454
455
456
457
459
463
464
468
471
473
474
475
478
479
485
486
487
488
491
302
307
331
335
346
367
377
381
382
388
389
390
395
403
404
406
413
417
418
420
422
436
439
440
451
Empty
458
Empty
472
476
477
482
483
484
489
490
492


In [ ]:
np.savez('fuse_whole_samples_regression.npz', features_whole)
np.savez('fuse_whole_labels_regression.npz', targets_whole)

# Classification

In [4]:
features_train = np.load('fuse_train_samples_classification.npz', allow_pickle=True)['arr_0']
features_test = np.load('fuse_test_samples_classification.npz', allow_pickle=True)['arr_0']
targets_train = np.load('fuse_train_labels_classification.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('fuse_test_labels_classification.npz', allow_pickle=True)['arr_0']

X_train = np.array(features_train)
X_test = np.array(features_test)
Y_train = np.array(targets_train)
Y_test = np.array(ctargets_test)

In [7]:
for i in range(len(X_train)):
  X_train[i][1] = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train[i][1]]).astype('float32')

for i in range(len(X_test)):
  X_test[i][1] = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test[i][1]]).astype('float32')

In [23]:
train_y = to_categorical(Y_train)
test_y = to_categorical(Y_test)

In [11]:
text_features = []
audio_features = []
for i in range(len(X_train)):
  text_features.append(X_train[i][0])
  audio_features.append(X_train[i][1])

text_features_test = []
audio_features_test = []
for i in range(len(X_test)):
  text_features_test.append(X_test[i][0])
  audio_features_test.append(X_test[i][1])

text_features = np.array(text_features)
audio_features = np.array(audio_features)

text_features_test = np.array(text_features_test)
audio_features_test = np.array(audio_features_test)

In [3]:
class AttentionLayer_1(tf.keras.Model):
    def __init__(self, hidden_dims):
        super(AttentionLayer_1, self).__init__()

        self.linear_layer = Dense(hidden_dims)
        self.relu_layer = tf.keras.layers.ReLU()

    def call(self, inputs):
        x = self.linear_layer(inputs)
        x = self.relu_layer(x)
        return x


def attention(output, final_hidden_state):
    lstm_hidden = tf.expand_dims(tf.convert_to_tensor(final_hidden_state), 1)
    attention_layer = AttentionLayer_1(hidden_dims=128)
    atten_w = attention_layer(lstm_hidden)
    tanh = tf.keras.activations.tanh(output)
    c = tf.matmul(atten_w, tf.transpose(tanh, (0, 2, 1)))
    softmax_w = tf.keras.activations.softmax(c, axis=-1)
    context = tf.matmul(softmax_w, output)
    result = tf.squeeze(context, axis=1)
    return result

input1 = Input(shape=(80, 469, 1))
x1 = Conv2D(32, (1, 7), activation='relu', input_shape=(80, 469, 1))(input1)
x1 = MaxPooling2D(pool_size=(4, 3), strides=(1, 3))(x1)
x1 = Conv2D(32, (1, 7), activation='relu')(x1)
x1 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(x1)
x1 = Flatten()(x1)
x1 = Dense(128, activation='relu')(x1)
x1 = Dense(128, activation='relu')(x1)

input2 = Input(shape=(10, 1024))

forward_layer = LSTM(128,  activation='relu', dropout=0.5, return_sequences=True, return_state=True)
backward_layer = LSTM(128, activation='relu',  dropout=0.5, return_sequences=True, go_backwards=True, return_state=True)
bidir = Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(10, 1024), merge_mode='sum')

output, forward_state, forward_sell_tate, backward_state, backward_sell_state = bidir(input2)
final_hidden_state = forward_state + backward_state

x2 = attention(output, final_hidden_state)

x2 = Dropout(0.5)(x2)
x2 = Dense(128, activation='relu')(x2)
x2 = Dropout(0.5)(x2)
x2 = Dense(128, activation='relu')(x2)

merged = Concatenate(axis=1)([x1, x2])
outputs = tf.keras.layers.Dense(2, activation='softmax')(merged)

In [4]:
model = Model(inputs=[input1, input2], outputs=outputs)

In [71]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])
model.fit([audio_features, text_features], train_y, validation_data=([audio_features_test, text_features_test], test_y), epochs=10, batch_size=2)

Epoch 1/10
76/76 [==============================] - 8s 38ms/step - loss: 0.7237 - accuracy: 0.4079 - val_loss: 0.6818 - val_accuracy: 0.5455
Epoch 2/10
76/76 [==============================] - 2s 31ms/step - loss: 0.6780 - accuracy: 0.5329 - val_loss: 0.6645 - val_accuracy: 0.6061
Epoch 3/10
76/76 [==============================] - 2s 24ms/step - loss: 0.6162 - accuracy: 0.7368 - val_loss: 0.6945 - val_accuracy: 0.5152
Epoch 4/10
76/76 [==============================] - 2s 25ms/step - loss: 0.5596 - accuracy: 0.7895 - val_loss: 0.6662 - val_accuracy: 0.6364
Epoch 5/10
76/76 [==============================] - 3s 35ms/step - loss: 0.4891 - accuracy: 0.8553 - val_loss: 0.6847 - val_accuracy: 0.6061
Epoch 6/10
76/76 [==============================] - 2s 31ms/step - loss: 0.4164 - accuracy: 0.9013 - val_loss: 0.6622 - val_accuracy: 0.5455
Epoch 7/10
76/76 [==============================] - 2s 24ms/step - loss: 0.3956 - accuracy: 0.8553 - val_loss: 0.6639 - val_accuracy: 0.5455
Epoch 8/10
76

In [72]:
y_pred = model.predict([audio_features_test, text_features_test])

2/2 [==============================] - 0s 11ms/step


In [73]:
predicted_1 = [1 if x[1] > x[0] else 0 for x in y_pred]
print(classification_report(Y_test, predicted_1))
print(confusion_matrix(Y_test, predicted_1))

              precision    recall  f1-score   support

           0       0.80      0.57      0.67        21
           1       0.50      0.75      0.60        12

    accuracy                           0.64        33
   macro avg       0.65      0.66      0.63        33
weighted avg       0.69      0.64      0.64        33

[[12  9]
 [ 3  9]]


In [74]:
model.save("fusion_classification_model.keras")

# Regression

In [88]:
features_train = np.load('fuse_train_samples_regression.npz', allow_pickle=True)['arr_0']
features_test = np.load('fuse_test_samples_regression.npz', allow_pickle=True)['arr_0']
targets_train = np.load('fuse_train_labels_regression.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('fuse_test_labels_regression.npz', allow_pickle=True)['arr_0']

X_train = np.array(features_train)
X_test = np.array(features_test)
Y_train = np.array(targets_train)
Y_test = np.array(ctargets_test)

In [89]:
for i in range(len(X_train)):
  X_train[i][1] = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train[i][1]]).astype('float32')

for i in range(len(X_test)):
  X_test[i][1] = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test[i][1]]).astype('float32')

In [98]:
text_features = []
audio_features = []
for i in range(len(X_train)):
  text_features.append(X_train[i][0])
  audio_features.append(X_train[i][1])

text_features_test = []
audio_features_test = []
for i in range(len(X_test)):
  text_features_test.append(X_test[i][0])
  audio_features_test.append(X_test[i][1])

text_features = np.array(text_features)
audio_features = np.array(audio_features)

text_features_test = np.array(text_features_test)
audio_features_test = np.array(audio_features_test)

In [ ]:
class AttentionLayer_1(tf.keras.Model):
    def __init__(self, hidden_dims):
        super(AttentionLayer_1, self).__init__()

        self.linear_layer = Dense(hidden_dims)
        self.relu_layer = tf.keras.layers.ReLU()

    def call(self, inputs):
        x = self.linear_layer(inputs)
        x = self.relu_layer(x)
        return x


def attention(output, final_hidden_state):
    lstm_hidden = tf.expand_dims(tf.convert_to_tensor(final_hidden_state), 1)
    attention_layer = AttentionLayer_1(hidden_dims=128)
    atten_w = attention_layer(lstm_hidden)
    tanh = tf.keras.activations.tanh(output)
    c = tf.matmul(atten_w, tf.transpose(tanh, (0, 2, 1)))
    softmax_w = tf.keras.activations.softmax(c, axis=-1)
    context = tf.matmul(softmax_w, output)
    result = tf.squeeze(context, axis=1)
    return result

input1 = Input(shape=(80, 469, 1))
x1 = Conv2D(32, (1, 7), activation='relu', input_shape=(80, 469, 1))(input1)
x1 = MaxPooling2D(pool_size=(4, 3), strides=(1, 3))(x1)
x1 = Conv2D(32, (1, 7), activation='relu')(x1)
x1 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(x1)
x1 = Flatten()(x1)
x1 = Dense(128, activation='relu')(x1)
x1 = Dense(128, activation='relu')(x1)

input2 = Input(shape=(10, 1024))

forward_layer = LSTM(128,  activation='relu', dropout=0.5, return_sequences=True, return_state=True)
backward_layer = LSTM(128, activation='relu',  dropout=0.5, return_sequences=True, go_backwards=True, return_state=True)
bidir = Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(10, 1024), merge_mode='sum')

output, forward_state, forward_sell_tate, backward_state, backward_sell_state = bidir(input2)
final_hidden_state = forward_state + backward_state

x2 = attention(output, final_hidden_state)

x2 = Dropout(0.5)(x2)
x2 = Dense(128, activation='relu')(x2)
x2 = Dropout(0.5)(x2)
x2 = Dense(128, activation='relu')(x2)

merged = Concatenate(axis=1)([x1, x2])
outputs = tf.keras.layers.Dense(1, activation='linear')(merged)

In [122]:
model = Model(inputs=[input1, input2], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit([audio_features, text_features], Y_train, validation_data=([audio_features_test, text_features_test], Y_test), epochs=20, batch_size=2)

Epoch 1/20
76/76 [==============================] - 7s 36ms/step - loss: 50.3197 - mean_absolute_error: 5.6373 - val_loss: 44.3904 - val_mean_absolute_error: 5.6880
Epoch 2/20
76/76 [==============================] - 3s 33ms/step - loss: 35.2082 - mean_absolute_error: 4.8067 - val_loss: 47.5834 - val_mean_absolute_error: 5.9423
Epoch 3/20
76/76 [==============================] - 2s 23ms/step - loss: 28.3283 - mean_absolute_error: 4.2968 - val_loss: 43.8997 - val_mean_absolute_error: 5.6218
Epoch 4/20
76/76 [==============================] - 2s 24ms/step - loss: 22.6990 - mean_absolute_error: 3.8631 - val_loss: 41.4916 - val_mean_absolute_error: 5.3305
Epoch 5/20
76/76 [==============================] - 2s 23ms/step - loss: 15.6606 - mean_absolute_error: 3.0758 - val_loss: 43.0306 - val_mean_absolute_error: 5.2876
Epoch 6/20
76/76 [==============================] - 2s 23ms/step - loss: 11.3086 - mean_absolute_error: 2.5945 - val_loss: 40.9592 - val_mean_absolute_error: 5.4433
Epoch 7/20

In [115]:
loss, mean_absolute_error = model.evaluate([audio_features_test, text_features_test], Y_test)
print(f"Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}")
y_pred = model.predict([audio_features_test, text_features_test])
print(np.sqrt(mean_squared_error(Y_test,y_pred)))

2/2 [==============================] - 0s 13ms/step - loss: 35.8677 - mean_absolute_error: 5.0579
Test Loss: 35.86766052246094, Test Mean Absolute Error: 5.057867527008057


2/2 [==============================] - 0s 12ms/step
5.988961437922379
